In [ ]:
import requests, requests.models, jwt, socket, random, string, time, base64, hashlib
from contextlib import closing
from pprint import pprint
from urllib.parse import urlparse, parse_qs
from contextlib import contextmanager
from threading import Thread


CATALOG_CLIENT_ID = "iceberg-catalog"
AUTH_URL = "http://localhost:30080/realms/iceberg/protocol/openid-connect/auth"
TOKEN_URL = "http://localhost:30080/realms/iceberg/protocol/openid-connect/token"
DEVICE_URL = "http://localhost:30080/realms/iceberg/protocol/openid-connect/auth/device"

# Human Authentication

Human Authentication flows are interactive by nature and are typically performed directly by the IdP. This enables the use of the full set of security options that the IdP supports, including 2FA, Hardware Keys and more. Widely supported flows include the Authorization Code Flow (RFC6749#section-4.1), which requires the User Agent (i.e. Spark) to open a web socket to receive the callback with the credentials. If this is not possible, a Device Code Flow (RFC8628) can be used, which only needs the option to present the user a short code.

## Option 1: Authorization Code Flow (RFC6749#section-4.1) with Proof Key of Code Exchange
Authorization Code Flow requires an open port which can be reached from the clients browser to recieve the credentials via a `/callback`. If this is not possible, clients can use Option 2: Device Code flow.

### A: Client-Side
When prompted, use 
* username: `peter`
* password: `iceberg`

In [ ]:
def find_free_port():
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
        s.bind(("", 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]


def random_alphanum_string(length: int):
    return "".join(random.choices(string.ascii_letters + string.digits, k=length))


def oneshot_reciever(bind_host: str, bind_port: int, listen_path: str):
    """Spawn a lightweight webserver, listening on GET to <bind_host>:<bind_port> for one request."""
    from http.server import BaseHTTPRequestHandler, HTTPServer
    from dataclasses import dataclass, field
    from typing import Any

    @dataclass
    class OneShotRetriever:
        _oneshot_done: bool = field(init=True, repr=False, default=False)
        result: None | Any = None
        server: Any | None = None

        @property
        def oneshot_done(self) -> str:
            return self._oneshot_done

        @oneshot_done.setter
        def oneshot_done(self, value: bool) -> None:
            self._oneshot_done = value
            self.server.server_close()

    retriever = OneShotRetriever()

    class CallbackReciever(BaseHTTPRequestHandler):
        def do_GET(self):
            full_path = urlparse(self.path)
            query = parse_qs(full_path.query)
            retriever.oneshot_done = True

            if "error" in query:
                print(f'ERROR: {query["error"]}: {query.get("error_description")}')

            if full_path.path != listen_path:
                self.send_response(404)
                self.send_header("Content-type", "text/html")
                self.end_headers()
                self.wfile.write("Not found".encode("utf-8"))
            else:
                # ... More error handling needed
                self.send_response(200)
                self.send_header("Content-type", "text/html")
                self.end_headers()
                self.wfile.write("OK".encode("utf-8"))
                retriever.result = query["code"][0]

    httpd = HTTPServer((bind_host, bind_port), CallbackReciever)
    thread = Thread(target=httpd.serve_forever).start()
    retriever.server = httpd

    return retriever

In [ ]:
oauth2_properties = {
    "auth-endpoint": AUTH_URL,
    "token-endpoint": TOKEN_URL,
    "scope": CATALOG_CLIENT_ID,
    "auth-code.callback-bind-port": 34987,
    "auth-code.callback-bind-host": "0.0.0.0",
    "auth-code.callback-path": "/iceberg-rest-client/callback",
    "auth-code.callback-url": "http://localhost:34987/iceberg-rest-client/callback",
    "client-id": CATALOG_CLIENT_ID,
    "grant-type": "authorization_code",
}

# ---------------- Handled in Iceberg Package ----------------
scope = oauth2_properties.get("scope", "catalog")
callback_path = oauth2_properties.get(
    "auth-code.callback-path", "/iceberg-rest-client/callback"
)
bind_port = oauth2_properties.get("auth-code.callback-bind-port") or find_free_port()
bind_host = oauth2_properties.get("auth-code.callback-bind-host", "localhost")
callback_uri = oauth2_properties.get(
    "auth-code.redirect-url", f"http://{bind_host}:{bind_port}{callback_path}"
)

if "localhost" == urlparse(callback_uri).netloc.split(":")[0]:
    print("WARNING: localhost redirect is not concidered secure!")

# Prepare PKCE for public client (no client-secret specified)
pkce_code_verifier = random_alphanum_string(128)
pkce_code_challenge = hashlib.sha256(pkce_code_verifier.encode("utf-8")).digest()
pkce_code_challenge = (
    base64.urlsafe_b64encode(pkce_code_challenge).decode("utf-8").replace("=", "")
)

required_params = {}
optional_params = {}

if scope is not None:
    optional_params["scope"] = scope

authorization_uri = requests.models.PreparedRequest()
authorization_uri.prepare_url(
    oauth2_properties.get("auth-endpoint"),
    params={
        "response_type": "code",
        "client_id": oauth2_properties["client-id"],
        "redirect_uri": callback_uri,
        "code_challenge": pkce_code_challenge,
        "code_challenge_method": "S256",
        **optional_params,
    },
)

# Start a mini webserver to recieve the callback request
reciever = oneshot_reciever(bind_host, bind_port, listen_path=callback_path)
# Clients may also try to open a new browser window for the user:
print(f"Please open the following URL and authenticate: {authorization_uri.url}")

while not reciever.oneshot_done:
    print("Waiting for human to complete auth (user: peter, pw: iceberg) ...")
    time.sleep(2)

if reciever.result is None:
    raise ValueError("Something weng wrong while getting Authorization Code")
else:
    print("Well done human!")

authorization_code = reciever.result

# Exchange code for token
response = requests.post(
    url=oauth2_properties["token-endpoint"].replace(
        "localhost:30080", "keycloak:8080"
    ),  # Replace only required due to docker setup
    data={
        "grant_type": "authorization_code",
        "code": authorization_code,
        "client_id": oauth2_properties["client-id"],
        "redirect_uri": callback_uri,
        "code_verifier": pkce_code_verifier,
    },
    headers={"Host": "localhost:30080"},
    allow_redirects=False,
)
response.raise_for_status()

access_token = response.json()["access_token"]
refresh_token = response.json()["refresh_token"]

# ... Implement background process to periodicly refresh the token.

In [ ]:
# Refresh
response = requests.post(
    url=oauth2_properties["token-endpoint"].replace(
        "localhost:30080", "keycloak:8080"
    ),  # Replace only required due to docker setup
    data={
        "grant_type": "refresh_token",
        "client_id": oauth2_properties["client-id"],
        "refresh_token": refresh_token,
    },
    headers={"Host": "localhost:30080"},
    allow_redirects=False,
)
response.raise_for_status()

### B: Server Side

In [ ]:
# ------------------- Server side -------------------
# Identical to Machine-to-Machine :)
TOKEN_INTROSPECTION_URI = (
    "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token/introspect"
)
CATALOG_AUTH_CHECK_CLIENT_ID = "iceberg-catalog-authenticator"
CATALOG_AUTH_CHECK_CLIENT_SECRET = "UDeCLaqjSisBBcL8h4JctCXcXdP9f0Jo"

# 1. Validate token - there are different ways to do this including token introspection and local JWT introspection
# We are using token introspection endpoint as an example here, because it allows the use of opaque tokens as well.
# If only jwt tokens are used, local jwks validation is typically more performant but doens't offer point-in-time logout.
response = requests.post(
    url="http://keycloak:8080/realms/iceberg/protocol/openid-connect/token/introspect",
    data={"token": access_token},
    headers={
        "Content-type": "application/x-www-form-urlencoded",
        "Host": "localhost:30080",
    },  # Specifying the Host here is only required due to our docker setup
    auth=(CATALOG_AUTH_CHECK_CLIENT_ID, CATALOG_AUTH_CHECK_CLIENT_SECRET),
)
response.raise_for_status()

# 2. Check Audience / Scope / Resouce
# Depending on your specific setup, check if the token is intended for this service by
# checking the scope, audience (aud) or resource field.
if CATALOG_CLIENT_ID in response.json()["aud"]:
    print(f"You may proceed {response.json()['name']} :)")
    print(f"Subject: {response.json()['sub']}")
else:
    print("Unauthenticated!")

## Option 2: Device Code Flow with PKCE
If opening a port is not possible, the Device Code Flow can be considered.

### A: Client Side

In [ ]:
oauth2_properties = {
    "token-endpoint": TOKEN_URL,
    "device-endpoint": DEVICE_URL,
    "scope": CATALOG_CLIENT_ID,
    "client-id": CATALOG_CLIENT_ID,
    "grant-type": "device_code",
}

scope = oauth2_properties.get("scope", "catalog")

optional_params = {}

if scope is not None:
    optional_params["scope"] = scope

pkce_code_verifier = random_alphanum_string(128)
pkce_code_challenge = hashlib.sha256(pkce_code_verifier.encode("utf-8")).digest()
pkce_code_challenge = (
    base64.urlsafe_b64encode(pkce_code_challenge).decode("utf-8").replace("=", "")
)

# Get device code
response = requests.post(
    url=oauth2_properties["device-endpoint"].replace(
        "localhost:30080", "keycloak:8080"
    ),  # Replace only required due to docker setup,
    data={
        "client_id": oauth2_properties["client-id"],
        "code_challenge_method": "S256",
        "code_challenge": pkce_code_challenge,
        **optional_params,
    },
    headers={
        "Content-type": "application/x-www-form-urlencoded",
        "Host": "localhost:30080",  # Specifying the Host here is only required due to our docker setup
    },
    allow_redirects=False,
)
response.raise_for_status()

verification_uri_complete = response.json()["verification_uri_complete"]
device_code = response.json()["device_code"]

# Present this to your user - i.e. via STDOUT, display, log, QR-Code or any other means available
print(f"Please open {verification_uri_complete}.")

while True:
    response = requests.post(
        url=oauth2_properties["token-endpoint"].replace(
            "localhost:30080", "keycloak:8080"
        ),  # Replace only required due to docker setup,
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
            "client_id": oauth2_properties["client-id"],
            "device_code": device_code,
            "code_verifier": pkce_code_verifier,
        },
        headers={
            "Content-type": "application/x-www-form-urlencoded",
            "Host": "localhost:30080",
        },  # Specifying the Host here is only required due to our docker setup
        allow_redirects=False,
    ).json()
    if "error" in response and response["error"] == "authorization_pending":
        print("Waiting for human to complete auth (user: peter, pw: iceberg) ...")
        time.sleep(5)
        continue
    else:
        print("Finished!")
        break

access_token = response["access_token"]
refresh_token = response["refresh_token"]

### B: Server Side

In [ ]:
# ------------------- Server side -------------------
TOKEN_INTROSPECTION_URI = (
    "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token/introspect"
)
CATALOG_AUTH_CHECK_CLIENT_ID = "iceberg-catalog-authenticator"
CATALOG_AUTH_CHECK_CLIENT_SECRET = "UDeCLaqjSisBBcL8h4JctCXcXdP9f0Jo"

# 1. Validate token - there are different ways to do this including token introspection and local JWT introspection
# We are using token introspection endpoint as an example here, because it allows the use of opaque tokens as well.
# If only jwt tokens are used, local jwks validation is typically more performant but doens't offer point-in-time logout.
response = requests.post(
    url="http://keycloak:8080/realms/iceberg/protocol/openid-connect/token/introspect",
    data={"token": access_token},
    headers={
        "Content-type": "application/x-www-form-urlencoded",
        "Host": "localhost:30080",
    },  # Specifying the Host here is only required due to our docker setup
    auth=(CATALOG_AUTH_CHECK_CLIENT_ID, CATALOG_AUTH_CHECK_CLIENT_SECRET),
)
response.raise_for_status()

# 2. Check Audience / Scope / Resouce
# Depending on your specific setup, check if the token is intended for this service by
# checking the scope, audience (aud) or resource field.
if CATALOG_CLIENT_ID in response.json()["aud"]:
    print(f"You may proceed {response.json()['name']} :)")
    print(f"Subject: {response.json()['sub']}")
else:
    print("Unauthenticated!")